## Geospatial Analysis II:  buffers, cost surfaces, least cost path
Resources:

* [
GRASS GIS overview and manual](http://grass.osgeo.org/grass72/manuals/index.html)
*  [Recommendations](data_acquisition.html#commands)
and [tutorial](http://www4.ncsu.edu/~akratoc/GRASS_intro/)
how to use GUI from the first assignment



Text files with recode rules, color rules and site locations:

* [noise_cats.txt](data/noise_cats.txt)
* [friction_rules.txt](data/friction_rules.txt)
* [friction_color.txt](data/friction_color.txt)
* [fire_pt.txt](data/fire_pt.txt)
* [lostperson.txt](data/lostperson.txt)


### Start GRASS GIS
In startup pannel set GIS Data Directory to path to datasets,
for example on MS Windows, `C:\Users\myname\grassdata`.
For Project location select nc_spm_08_grass7 (North Carolina, State Plane, meters) and
for Accessible mapset create a new mapset (called e.g. HW_buffers_cost) and
click Start GRASS.

In [ ]:
# using Python to initialize GRASS GIS
# This is a quick introduction into Jupyter Notebook.
# Python code can be excecuted like this:
a = 6
b = 7
c = a * b
print "Answer is", c
# Python code can be mixed with command line code (Bash).
# It is enough just to prefix the command line with an exclamation mark:
!echo "Answer is $c"
# Use Shift+Enter to execute this cell. The result is below.

In [ ]:
# using Python to initialize GRASS GIS
import os
import sys
import subprocess
from IPython.display import Image

# create GRASS GIS runtime environment
gisbase = subprocess.check_output(["grass", "--config", "path"]).strip()
os.environ['GISBASE'] = gisbase
sys.path.append(os.path.join(gisbase, "etc", "python"))

# do GRASS GIS imports
import grass.script as gs
import grass.script.setup as gsetup

# set GRASS GIS session data
rcfile = gsetup.init(gisbase, "/home/jovyan/grassdata", "nc_spm_08_grass7", "user1")

In [ ]:
# using Python to initialize GRASS GIS
# default font displays
os.environ['GRASS_FONT'] = 'sans'
# overwrite existing maps
os.environ['GRASS_OVERWRITE'] = '1'
gs.set_raise_on_error(True)
gs.set_capture_stderr(True)

In [ ]:
# using Python to initialize GRASS GIS
# set display modules to render into a file (named map.png by default)
os.environ['GRASS_RENDER_IMMEDIATE'] = 'cairo'
os.environ['GRASS_RENDER_FILE_READ'] = 'TRUE'
os.environ['GRASS_LEGEND_FILE'] = 'legend.txt'

Remove MASK in case it was left over
from previous assignment (use command `r.mask -r`).

Change working directory:
_Settings_ > _GRASS working environment_ > _Change working directory_ > select/create any directory
or type `cd` (stands for change directory) into the GUI
_Console_ and hit Enter:

In [ ]:
# a proper directory is already set, download files
import urllib
urllib.urlretrieve("http://ncsu-geoforall-lab.github.io/geospatial-modeling-course/grass/data/noise_cats.txt", "noise_cats.txt")
urllib.urlretrieve("http://ncsu-geoforall-lab.github.io/geospatial-modeling-course/grass/data/friction_rules.txt", "friction_rules.txt")
urllib.urlretrieve("http://ncsu-geoforall-lab.github.io/geospatial-modeling-course/grass/data/friction_color.txt", "friction_color.txt")
urllib.urlretrieve("http://ncsu-geoforall-lab.github.io/geospatial-modeling-course/grass/data/fire_pt.txt", "fire_pt.txt")
urllib.urlretrieve("http://ncsu-geoforall-lab.github.io/geospatial-modeling-course/grass/data/lostperson.txt", "lostperson.txt")
urllib.urlretrieve("http://ncsu-geoforall-lab.github.io/geospatial-modeling-course/grass/data/noise_cats.txt", "noise_cats.txt")
urllib.urlretrieve("http://ncsu-geoforall-lab.github.io/geospatial-modeling-course/grass/data/fire_pt.txt", "fire_pt.txt")
urllib.urlretrieve("http://ncsu-geoforall-lab.github.io/geospatial-modeling-course/grass/data/friction_rules.txt", "friction_rules.txt")
urllib.urlretrieve("http://ncsu-geoforall-lab.github.io/geospatial-modeling-course/grass/data/lostperson.txt", "lostperson.txt")

Download all text files with recode rules, color rules and site locations (see above)
to the selected directory. Now you can use the commands from the assignment requiring the text file
without the need to specify the full path to the file.




### Buffers
#### Find developed areas close to lakes
Set region, create buffers:

In [ ]:
!g.region swwake_30m -p
!r.buffer lakes output=lakes_buff distances=60,120,240,500
!d.rast lakes_buff
Image(filename="map.png")

List categories in land use map to identify cat numbers for developed areas.
Then run _r.mapcalc_:

In [ ]:
!r.category landuse96_28m
!r.mapcalc "developed_lake = if(landuse96_28m==1 || landuse96_28m==2, lakes_buff, null())"
!r.support developed_lake raster=lakes_buff

Display results:
In GUI: switch off previous map, use Add raster, Add vector icons.

In [ ]:
!d.rast developed_lake
!d.vect streets_wake color=grey
!d.rast lakes
!d.legend -c developed_lake use=2,3,4,5 at=5,25,2,5
Image(filename="map.png")

Find the total area within buffers and the developed area in ha:

In [ ]:
!r.report -n lakes_buff units=h
!r.report -n developed_lake units=h

#### Find developed areas impacted by noise from highways
Set region and create buffers along major roads.
Intersect developed areas from landuse map with road buffers.
Transfer the category labels and compute the affected area.
Run _r.category_ from GUI and browse to the txt file,
or run _r.category_ command (assuming [noise_cats.txt](data/noise_cats.txt)
is in your working directory).

In [ ]:
!g.region raster=landuse96_28m -p
!r.buffer roadsmajor output=roads_buffers distances=250,500,2500
!r.mapcalc "noise = if(landuse96_28m==1 || landuse96_28m==2, roads_buffers, null())"
!r.colors noise color=ryg
!r.category noise rules=noise_cats.txt separator=:
!r.report -n noise units=p,h

Display the results. In GUI switch off previous maps.

In [ ]:
!d.rast noise
!d.vect streets_wake
!d.legend -c noise at=5,25,2,5
Image(filename="map.png")

#### Find schools affected by high levels of noise
Convert schools to raster using CORECAPACI attribute (school capacity).
Use map algebra to overlay with noise impact buffers and
compute the number of students exposed to noise (see result of _r.univar_).

In [ ]:
!v.to.rast schools_wake output=schoolscap_10m use=attr attrcolumn=CORECAPACI type=point
!r.mapcalc "schools_noise = if(int(schoolscap_10m) && roads_buffers == 2, int(schoolscap_10m), null())"
!r.univar schools_noise
!r.to.vect schools_noise output=schools_noise type=point
!d.vect schools_wake icon=basic/circle size=10 fill_color=blue
!d.vect schools_noise size=14 fill_color=cyan icon=basic/circle color=black
Image(filename="map.png")

### Cost surfaces

#### Compute cumulative cost surface to a given accident site based on speed limits

Use 5mi/hr speed limit for off-road areas (nulls).
For legend set the discrete speed limit values in GUI legend dialog:
tab _Required_ > streets_speed, _Subset_ > _List of discrete category numbers_

In [ ]:
!g.region swwake_30m -p
!v.info -c streets_wake
!v.to.rast streets_wake output=streets_speedtmp use=attr attrcolumn=SPEED type=line
!r.mapcalc "streets_speed = if(isnull(streets_speedtmp),5,streets_speedtmp)"
!r.info streets_speed
!r.colors streets_speed color=gyr
!d.rast streets_speed
!d.legend streets_speed at=5,40,2,5 use=5,25,35,45,65
Image(filename="map.png")

Assign travel time to cross a 30m grid cell in hours.
Note that cost in GRASS GIS is defined as travel time per _cell_.

Compute cumulative cost surface to the given point.
Where does the 0.018641 constant come from?
You can modify the expression to get time in minutes.

In [ ]:
!r.mapcalc "streets_travtime = 0.018641/streets_speed"
!r.cost -k streets_travtime output=streets_cost start_coordinates=634886,224328

Import the accident point location given in [fire_pt.txt](data/fire_pt.txt)
so that you can display it on the map.
Run _v.in.ascii_ from GUI, use file browser to provide full path to the file
or enter values interactively by pasting the coords into the provided space,
or run the command assuming the file is in your working directory.

In [ ]:
!v.in.ascii input=fire_pt.txt output=fire_pt separator=space

Compute isochrones and display the cumulative cost surface map.

In [ ]:
!r.contour streets_cost output=streets_cost_04 step=0.04
!r.colors streets_cost color=elevation
!d.rast streets_cost
!d.vect fire_pt color=red icon=basic/marker size=20
!d.vect streets_cost_04
!d.legend streets_cost at=5,50,2,5
Image(filename="map.png")

#### Find cost (travel time) from selected firestations

First make your own copy of the firestations map and list attributes.
Then query the cumulative cost surface at the firestations location.
The travel time in hours will be stored in the attribute column CVLAG.

In [ ]:
!g.copy vector=firestations,myfirestations
!v.info -c myfirestations
!v.what.rast myfirestations raster=streets_cost column=CVLAG

Add myfirestations to Layer Manager and right click on Show attribute data.
Click on CVLAG to order them and find the lowest cost (shortest time) > 0
(firestations with 0 cost are outside the region).
You should see a [table like this](img/firestations_table.png).
Export firestations with traveltime less than 0.1 hr:

In [ ]:
!v.out.ascii input=myfirestations separator=space precision=3 columns=ID,LOCATION,CVLAG where="CVLAG<0.1 AND CVLAG>0"

You should get something like the following:
```
635775.565 228121.693 19 19 4021 District Dr 0.076
635940.262 225912.796 20 0 5001 Western Blvd 0.037
637386.831 222569.152 21 0 1721 Trailwood Dr 0.071
633178.155 221353.037 52 27 6000 Holly Springs Rd 0.060
```

Find the least cost path for the two closest stations:

In [ ]:
!r.drain -n input=streets_cost output=route_20Westernb start_coordinates=635940.3,225912.8
!r.drain -n input=streets_cost output=route_52Hollyb start_coordinates=633178.2,221353.0
!r.colors route_20Westernb color=grey
!r.colors route_52Hollyb color=grey

Display the results in GUI.
You already may have some of these maps in GUI layer tree,
so just switch them on and off and re-order as needed:

In [ ]:
!d.erase
!d.rast streets_cost
!d.vect fire_pt color=red icon=basic/marker size=20
!d.rast route_20Westernb
!d.rast route_52Hollyb
Image(filename="map.png")

Print the length of the path in cells (multiply by 30m to get approx. m).
You should have the time in hr already from the cost map.
At what average speed [km/hr] needs the truck travel to get there in estimated time?
Is the time, speed and distance realistic?

In [ ]:
!r.describe route_20Westernb
!r.describe route_52Hollyb

If you would like to display streets_cost surface in 3D as shown in the lecture,
you need to multiply the surface by 1000 using _r.mapcalc_ because the numbers
in hours are very low.
Then switch off all layers except for the streets_cost_1000
and change display to 3D.

#### Compute accessibility map for help in search for lost person
Create friction map based on land cover.
First display land cover classes:

In [ ]:
!g.region swwake_30m -p
!r.category landclass96

Recode the landuse map to friction map using the rules in 
[friction_rules.txt](data/friction_rules.txt).

In [ ]:
!r.recode landclass96 out=friction rules=friction_rules.txt

Add the streets to friction map - they are missed by the landuse map.

In [ ]:
!r.mapcalc "friction2 = if(streets_speed > 6, 0.1, friction)"
!r.colors friction2 rules=friction_color.txt
!d.erase
!d.rast friction2
!d.legend friction2 at=5,50,2,5
Image(filename="map.png")

Reminder: Time to time, you should remove or at least uncheck previously
used map layers in the _Layer Manager_, so the layers are not rendered
when you don't need them anymore.


Compute the cost map and generate isochrones:

In [ ]:
!r.walk -k elevation=elev_ned_30m friction=friction2 output=walkcost start_coordinates=635576,216485 lambda=0.5 max_cost=10000
!r.contour walkcost output=walkcost step=1000

To display the input, import coordinates of the point where the lost person was last seen
given in the file [lostperson.txt](data/lostperson.txt).

In [ ]:
!v.in.ascii input=lostperson.txt output=lostperson separator=comma

Display the results:

In [ ]:
!d.erase
!d.rast lakes
!d.rast walkcost
!d.vect streets_wake
!d.vect walkcost color=brown
!d.vect walkcost color=red where=level=6000 width=2
!d.vect lostperson size=20 color=black fill_color=black icon=basic/marker
!d.legend walkcost at=5,50,2,5
Image(filename="map.png")

#### Cost surfaces to line features
Compute the shortest distance to a major road.
Set region, convert vector road map to raster:

In [ ]:
!g.region swwake_30m -p
!v.to.rast roadsmajor output=roadsmajor use=val type=line

Compute the distance map and cost surface to highways:

In [ ]:
!r.mapcalc "area_one = 1"
!r.cost input=area_one output=dist_toroad start_rast=roadsmajor
!r.mapcalc "dist_meters = dist_toroad * (ewres() + nsres())/2."
!r.mapcalc "dist_class = int(dist_meters/500)"
!d.erase
!d.rast dist_class
!d.vect roadsmajor
Image(filename="map.png")
!d.vect streets_wake color=black
!d.vect roadsmajor color=red
Image(filename="map.png")

In [ ]:
# end the GRASS session
os.remove(rcfile)